In [87]:
import pandas as pd
%pylab inline

Populating the interactive namespace from numpy and matplotlib


C:\ProgramData\Anaconda3\lib\site-packages\IPython\core\magics\pylab.py:160: UserWarning: pylab import has clobbered these variables: ['mean', 'median', 'diff']
`%matplotlib` prevents importing * from pylab and numpy
  "\n`%matplotlib` prevents importing * from pylab and numpy"


In [88]:
df = pd.read_excel(r"C:\Users\samas\Downloads\ECGDUSq4.xlsx")

In [89]:
df1 = pd.read_csv(r"C:\Users\samas\OneDrive\Desktop\spf.csv")

In [90]:
df1['cqtr'] =  4

In [91]:
df1['cyr'] = pd.to_datetime(df1.Deadline).dt.year
df1['cyr'] = df1['cyr'] + 1

In [92]:
df1['cyr'] = df1['cyr'].astype(str).str[-2:]

In [93]:
df1.cqtr = df1.cqtr.astype(str)

In [94]:
df1['forecast'] = 'Q' + df1['cqtr'] + df1['cyr']

In [95]:
df1

,Deadline,str_date,cqtr,cyr,forecast
0,02-12-2002,02-12-2002,4,03,Q403
1,05-13-2002,05-13-2002,4,03,Q403
2,08-14-2002,08-14-2002,4,03,Q403
3,11-13-2002,11-13-2002,4,03,Q403
4,02-14-2003,02-14-2003,4,04,Q404
...,...,...,...,...,...
75,11-10-2020,11-10-2020,4,21,Q421
76,02-09-2021,02-09-2021,4,22,Q422
77,05-12-2021,05-12-2021,4,22,Q422
78,08-10-2021,08-10-2021,4,22,Q422


In [96]:
df1['quarter'] = pd.to_datetime(df1['Deadline']).dt.quarter

In [97]:
df1 = df1[df1['quarter']==4]

In [98]:
df1.reset_index(inplace = True)

In [99]:
df1

,index,Deadline,str_date,cqtr,cyr,forecast,quarter
0,3,11-13-2002,11-13-2002,4,03,Q403,4
1,7,11-14-2003,11-14-2003,4,04,Q404,4
2,11,11-13-2004,11-13-2004,4,05,Q405,4
3,15,11-08-2005,11-08-2005,4,06,Q406,4
4,19,11-08-2006,11-08-2006,4,07,Q407,4
5,23,11-07-2007,11-07-2007,4,08,Q408,4
6,27,11-10-2008,11-10-2008,4,09,Q409,4
7,31,11-10-2009,11-10-2009,4,10,Q410,4
8,35,11-09-2010,11-09-2010,4,11,Q411,4
9,39,11-08-2011,11-08-2011,4,12,Q412,4


In [100]:
df['cqtr'] =  pd.to_datetime(df.Date).dt.quarter

In [101]:
df['str_date'] = df['Date'].astype(str)

In [102]:
df[(df['Date'].dt.year == 2005) & (df['forecast'] == 'Q106')& (df['Date'] < '2005-02-09')]

,Unnamed: 0,Date,Value,forecast,forecaster,cqtr,str_date


In [103]:
import statistics
new_data = []
dead_lines = []
qtr = []
elems = []
ninds = []
avg_days_diff = []
for idx, row in df1.iterrows():
    ld = pd.date_range(end = row['str_date'] , periods= 30, closed = 'left')
    ld = list(ld.sort_values(ascending = False).astype(str))
    dist = []
    forecast = []
    inds = []
    days_diff = []
    for i in ld:
        for idx1, row1 in df.iterrows():
            a = row1['str_date']
            if i == a and row['forecast'] == row1['forecast']:
                b = row1['Value']
                dist.append(b)
                c = row1['forecast']
                forecast.append(c)
                d = row1['forecaster']
                inds.append(d)
                d1 = pd.to_datetime(row['str_date'])
                d2 = row1['Date']
                diff = d1 - d2
                days_diff.append(diff.days)
            else:
                pass
    if len(dist) > 3:
        avg_days_diff.append(numpy.mean(days_diff))
        ninds.append(len(inds))
        forecast = set(forecast)
        qtr.append(forecast)
        dead_lines.append(row['str_date'])
        mean = numpy.mean(dist)
        median = statistics.median(dist)
        per_25 = numpy.percentile(dist, 25)
        per_75 = numpy.percentile(dist, 75)
        per_60 = numpy.percentile(dist, 60)
        per_90 = numpy.percentile(dist, 90)
        per_10 = numpy.percentile(dist, 10)
        per_40 = numpy.percentile(dist, 40)
        elems.append(dist)
        stats = [mean] + [median] + [per_25] + [per_75] + [per_60] + [per_90] + [per_10] + [per_40]
        new_data.append(stats)
    else:
        pass

In [104]:
len(new_data)

15

In [105]:
new_df = pd.DataFrame(new_data)

In [106]:
new_df.rename(columns = {0:'Mean', 1:'Median',2:'25th_percentile',3:'75th_percentile',4:'60th_percentile',5:'90th_percentile',6:'10th_percentile',7:'40th_percentile'}, inplace = True)
new_df['forecast'] = qtr
new_df['SPF_deadlines'] = dead_lines
new_df['no_of_forecasters'] = ninds
new_df['avg_days_diff'] = avg_days_diff

In [107]:
new_df

,Mean,Median,25th_percentile,75th_percentile,60th_percentile,90th_percentile,10th_percentile,40th_percentile,forecast,SPF_deadlines,no_of_forecasters,avg_days_diff
0,3.685714,3.600,3.250,4.3500,3.96,4.700,2.60,3.540,{Q404},11-14-2003,7,10.000000
1,3.157143,3.200,3.000,3.4000,3.26,3.500,2.84,3.080,{Q405},11-13-2004,7,4.000000
2,2.614286,2.500,2.500,2.6500,2.56,2.820,2.50,2.500,{Q408},11-07-2007,7,14.142857
3,3.015385,3.000,2.425,3.4750,3.00,4.100,1.95,2.900,{Q411},11-09-2010,26,27.000000
4,2.435484,2.400,1.850,2.7500,2.50,3.600,1.70,2.300,{Q412},11-08-2011,31,26.387097
5,2.740476,2.550,2.400,3.0000,2.80,3.690,2.01,2.500,{Q413},11-06-2012,42,24.642857
6,2.985294,3.000,2.625,3.2000,3.00,3.500,2.40,2.820,{Q414},11-18-2013,34,4.823529
7,2.727273,2.500,2.500,3.0000,3.00,3.000,2.50,2.500,{Q415},11-10-2014,11,12.636364
8,2.520000,2.500,2.500,2.5000,2.50,2.560,2.50,2.500,{Q416},11-10-2015,5,16.400000
9,1.887500,2.000,1.700,2.1000,2.10,2.500,1.50,2.000,{Q417},11-08-2016,48,23.208333


In [21]:
new_df['avg_days_diff'].mean()

20.563962425250946

In [108]:
new_df.to_csv(r"C:\Users\samas\Downloads\Quarter_4_forecasts.csv")

In [30]:
len(df['forecaster'].unique())

80

In [154]:
inds

['DB',
 'GS',
 'NTE',
 'SCI',
 'SG',
 'SWR',
 'ML',
 'LB',
 'ABN',
 'DB',
 'GS',
 'JPM',
 'SCI',
 'SG',
 'SWR',
 'ML',
 'ABN',
 'GS',
 'JPM',
 'MS',
 'NTE',
 'SWR',
 'ML',
 'ABN',
 'DB',
 'JPM',
 'NTE',
 'SCI',
 'SWR',
 'ABN',
 'DB',
 'JPM',
 'NTE',
 'SCI',
 'SG',
 'SWR',
 'LB',
 'GS',
 'JPM',
 'NTE',
 'SG',
 'SWR',
 'ML',
 'ABN',
 'GS',
 'NTE',
 'SCI',
 'SG',
 'SWR',
 'LB',
 'ABN',
 'DB',
 'GS',
 'JPM',
 'SG',
 'SWR',
 'LB',
 'ABN',
 'DB',
 'GS',
 'JPM',
 'MS',
 'SG',
 'SWR',
 'ML',
 'DB',
 'FTA',
 'GS',
 'JPM',
 'MS',
 'NTE',
 'SWR',
 'ML',
 'LB',
 'UCM',
 'SCB',
 'JPM',
 'SCB',
 'CB',
 'JBC',
 'DEK',
 'JBC',
 'DEK',
 'DEK',
 'JPM',
 'JPM',
 'DEK',
 'JPM',
 'JPM',
 'JPM',
 'JPM',
 'JPM',
 'WF',
 'MS',
 'JPM',
 'WF',
 'JPM',
 'MS',
 'JPM',
 'JPM',
 'JPM',
 'MS',
 'JPM',
 'NBF',
 'PPS',
 'BM',
 'BOA',
 'BTW',
 'BAR',
 'CIB',
 'CIT',
 'COM',
 'CB',
 'CAG',
 'CSU',
 'DES',
 'DB',
 'FTA',
 'GSU',
 'HLA',
 'HSB',
 'IG',
 'JPM',
 'MFA',
 'MEC',
 'MS',
 'NEA',
 'NS',
 'NRD',
 'OXE',
 'RJF',


In [47]:
new_df['Deadline'] = dead_lines

In [54]:
new_df.rename( columns = {0:'rn', 1:'Date', 2: 'Value', 3:'forecast', 4:'forecaster', 5:'str_date', 'Deadline':'SPF_deadline'}, inplace = True)

In [55]:
new_df

,rn,Date,Value,forecast,forecaster,str_date,SPF_deadline
0,15,2003-05-08,3.8,Q104,DB,2003-05-08,05-12-2003
1,17,2003-05-08,2.0,Q104,GS,2003-05-08,05-12-2003
2,13,2003-05-08,3.7,Q104,NTE,2003-05-08,05-12-2003
3,13,2003-05-08,3.2,Q104,SCI,2003-05-08,05-12-2003
4,12,2003-05-08,3.6,Q104,SG,2003-05-08,05-12-2003
...,...,...,...,...,...,...,...
646,8,2021-11-02,4.2,Q122,JBC,2021-11-02,11-09-2021
647,9,2021-10-29,4.5,Q122,BAR,2021-10-29,11-09-2021
648,9,2021-10-29,3.8,Q122,BGB,2021-10-29,11-09-2021
649,9,2021-10-29,3.5,Q122,JPM,2021-10-29,11-09-2021


In [82]:
date_qtr = df[['str_date','forecast']].apply(tuple, axis = 1).to_list()

In [83]:
date_qtr = set(date_qtr)

In [84]:
date_qtr

{('2018-11-30', 'Q118'),
 ('2017-10-12', 'Q117'),
 ('2011-03-11', 'Q111'),
 ('2013-12-10', 'Q114'),
 ('2016-05-12', 'Q117'),
 ('2017-04-13', 'Q118'),
 ('2017-06-29', 'Q118'),
 ('2020-01-31', 'Q121'),
 ('2007-10-24', 'Q108'),
 ('2021-03-05', 'Q120'),
 ('2015-03-12', 'Q116'),
 ('2019-07-19', 'Q119'),
 ('2015-04-09', 'Q115'),
 ('2016-02-08', 'Q116'),
 ('2017-07-28', 'Q117'),
 ('2005-05-09', 'Q106'),
 ('2021-04-29', 'Q121'),
 ('2016-06-17', 'Q117'),
 ('2015-01-16', 'Q115'),
 ('2015-07-30', 'Q116'),
 ('2016-02-26', 'Q117'),
 ('2010-11-15', 'Q110'),
 ('2016-07-22', 'Q117'),
 ('2018-03-16', 'Q118'),
 ('2018-11-23', 'Q118'),
 ('2016-04-29', 'Q116'),
 ('2016-09-09', 'Q117'),
 ('2016-09-08', 'Q118'),
 ('2015-07-31', 'Q115'),
 ('2015-10-08', 'Q117'),
 ('2021-04-13', 'Q121'),
 ('2012-08-09', 'Q114'),
 ('2008-03-10', 'Q109'),
 ('2021-04-09', 'Q121'),
 ('2017-03-09', 'Q118'),
 ('2021-11-12', 'Q122'),
 ('2019-05-06', 'Q120'),
 ('2019-10-25', 'Q119'),
 ('2021-04-09', 'Q122'),
 ('2017-01-17', 'Q117'),


In [70]:

for i in date_qtr:
    values = []
    qtr = i[1]
    
    for idx, row in df.iterrows():
        if row['str_date'] == date and row['forecast']== qtr:
            
            val = row['Value']
            values.append(val)
        else:
            pass            

[2.5]
('2015-10-30', 'Q116')
[3.5, 6.4]
('2021-04-30', 'Q121')
[0.8]
('2016-08-04', 'Q116')
[2.3]
('2016-05-06', 'Q117')
[2.6]
('2014-01-29', 'Q115')
[3.2]
('2017-05-05', 'Q118')
[2.4]
('2021-08-02', 'Q122')
[-2.1]
('2014-11-05', 'Q114')
[2.0, 2.5, 1.1]
('2020-01-31', 'Q121')
[1.5]
('2007-10-24', 'Q108')
[4.5, 3.5]
('2021-11-05', 'Q122')
[2.3]
('2016-01-28', 'Q117')
[0.8]
('2016-11-04', 'Q116')
[2.3]
('2016-04-29', 'Q117')
[2.3]
('2017-08-03', 'Q118')
[9.6]
('2021-01-29', 'Q122')
[-2.1]
('2014-07-30', 'Q114')
[3.1, 3.1, 3.1]
('2019-11-01', 'Q119')
[3.7]
('2010-11-02', 'Q110')
[-4.8, -4.8]
('2020-04-30', 'Q120')
[2.2]
('2009-02-06', 'Q110')
[0.5]
('2016-02-08', 'Q116')
[2.5, 2.2]
('2016-02-05', 'Q116')
[1.4]
('2017-07-28', 'Q117')
[2.5]
('2018-04-27', 'Q119')
[3.0, 3.9, 3.0, 3.5, 3.6, 3.2, 3.2]
('2005-05-09', 'Q106')
[2.0, 1.8, 0.7]
('2019-04-26', 'Q120')
[3.8, 5.6]
('2021-11-03', 'Q122')
[2.1]
('2018-08-06', 'Q119')
[4.2]
('2021-04-29', 'Q121')
[2.8]
('2017-11-02', 'Q118')
[2.6]
('2015

[2.2]
('2019-03-08', 'Q118')
[5.0, 4.0, 6.0]
('2020-08-07', 'Q121')
[2.5]
('2014-07-31', 'Q115')
[1.0]
('2008-07-29', 'Q109')
[2.0, 0.7]
('2019-05-03', 'Q120')
[2.4]
('2020-10-30', 'Q122')
[9.7]
('2021-02-04', 'Q122')
[7.5]
('2020-05-04', 'Q121')
[1.6]
('2013-02-06', 'Q113')
[2.3]
('2013-08-02', 'Q114')
[2.7, 2.9, 3.6, 2.2, 2.7, 2.5, 4.4, 2.9, 2.8, 3.1, 3.0, 3.2, 3.5, 3.0, 3.0, 2.5]
('2013-08-07', 'Q115')
[12.0, 8.5]
('2020-05-01', 'Q121')
[2.6]
('2015-11-05', 'Q116')
[2.5, 2.0, 1.9]
('2016-07-29', 'Q117')
[2.2, 2.2]
('2018-10-26', 'Q118')
[2.4]
('2020-11-06', 'Q122')
[2.5, 1.8, 1.5, 2.4]
('2019-03-08', 'Q119')
[2.3, 0.7]
('2019-03-08', 'Q120')
[2.8]
('2015-02-05', 'Q115')
[3.1, 3.1]
('2019-11-08', 'Q119')
[2.5, 2.4, 2.3, 1.84, 2.5]
('2018-10-26', 'Q119')
[2.5]
('2018-05-04', 'Q119')
[2.5]
('2014-02-07', 'Q114')
[2.0]
('2013-02-08', 'Q114')
[3.0, 2.7]
('2015-01-30', 'Q115')
[2.8]
('2017-10-26', 'Q118')
[2.7]
('2013-04-24', 'Q113')
[-2.5]
('2008-11-07', 'Q109')
[2.3]
('2021-02-05', 'Q12

,Unnamed: 0,Date,Value,forecast,forecaster,str_date
0,7,2020-04-10,1.9,Q121,ABN,2020-04-10
1,8,2020-04-09,1.9,Q121,ABN,2020-04-09
2,9,2020-02-14,2.2,Q121,ABN,2020-02-14
3,7,2020-04-10,-6.0,Q120,ABN,2020-04-10
4,8,2020-04-09,-6.0,Q120,ABN,2020-04-09
...,...,...,...,...,...,...
12026,11,2004-04-07,3.5,Q105,LB,2004-04-07
12027,7,2004-04-07,4.4,Q104,LB,2004-04-07
12028,8,2003-12-08,3.2,Q104,LB,2003-12-08
12029,9,2003-09-09,3.0,Q104,LB,2003-09-09


In [72]:
df1

,Deadline,str_date
0,02-12-2002,02-12-2002
1,05-13-2002,05-13-2002
2,08-14-2002,08-14-2002
3,11-13-2002,11-13-2002
4,02-14-2003,02-14-2003
...,...,...
75,11-10-2020,11-10-2020
76,02-09-2021,02-09-2021
77,05-12-2021,05-12-2021
78,08-10-2021,08-10-2021
